# Import

In [1]:
from torch import nn
import torch
from models import DrFuse_RS_Model, DrFuse_RS_Trainer
import time
import os
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm
import skimage.io as io
import earthpy.spatial as es
import numpy as np
import random
import wandb


# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

C:\Users\n10300759\AppData\Local\Temp\ipykernel_10968\2494540655.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Data

In [2]:
now = str(int(time.time()))

EXPERIMENT_CONFIG = dict(
    # window data folder
    DATA_FOLDER = r'D:\CarsSegmentationTroubleShoot\data\multimodal',
    # mac data folder
    # DATA_FOLDER = r'/Users/nhikieu/Documents/PhD/multimodal',
    num_epoch = 100, 
    img_size = 512, 
    num_classes = 6,
    input_channels = 5,
    batch_size = 4,
    checkpoint_pth = os.path.join('checkpoints', now),
    update_optim_size = 32
    )
DEVICE = 'mps'

print(EXPERIMENT_CONFIG['checkpoint_pth'])

checkpoints\1720658579


In [3]:
def rgb_to_class(mask_image):
  class_map = {
    (255,255,255): 0, 
    (0,0,255): 1, 
    (0,255,255): 2, 
    (0,255,0): 3, 
    (255,255,0): 4, 
    (255,0,0): 5
  }

  # Create a 3D numpy array that represents the RGB color of each pixel
  rgb_data = mask_image.reshape(-1, 3)

  # Create a 1D numpy array that represents the class label for each RGB color
  class_labels = np.zeros(rgb_data.shape[0], dtype=np.uint8)
  for rgb, class_label in class_map.items():
      mask = np.all(rgb_data == np.array(rgb), axis=1)
      class_labels[mask] = class_label

  # Reshape the 1D class label array into a 2D class map
  class_data = class_labels.reshape(mask_image.shape[:2])

  return class_data


class VaihingenDataset(Dataset):
  def __init__(self, folder) -> None:
    '''
    folder: data path include both rgb img and gt
    gt: 3 channels map with postfix '_gt'
    '''
    super().__init__()

    self.imgs = []
    self.gts = []

    # get all filenames in the directory
    filenames = os.listdir(os.path.join(folder, 'rgb'))

    for f in tqdm(filenames):
      img = io.imread(os.path.join(folder, 'rgb', f)) / 255.0
     
      ndsm = io.imread(os.path.join(folder, 'ndsm', f.split('.png')[0] + '_ndsm.tif'))
      ndsm = np.expand_dims(ndsm, axis=2)
    
      input_4C = np.dstack((img, ndsm))

      input_4C = torch.tensor(input_4C).permute((2, 0, 1))
      
      gt_path = os.path.join(folder, 'gt', f.split('.png')[0] + '_gt.png')
      gt = io.imread(gt_path)
      gt = rgb_to_class(gt)
      gt = torch.tensor(gt).unsqueeze(0)

      self.imgs.append(input_4C)
      self.gts.append(gt)
    

  def __len__(self):
    return len(self.imgs)
  
  def __getitem__(self, index):
    input_4C = self.imgs[index].float()
    gt = self.gts[index]

    # mask_array = [True, False]
    mask_index = np.random.choice(2, 1, p=[0.5, 0.5])
    mask = torch.tensor(mask_index[0])

    return input_4C, gt, mask

In [4]:
#Note that we are using ground truth in the folder '1CGT'. They are not RGB anymore, but one channel.
training_dataset = VaihingenDataset(os.path.join(EXPERIMENT_CONFIG['DATA_FOLDER'], 'train'))
validate_dataset = VaihingenDataset(os.path.join(EXPERIMENT_CONFIG['DATA_FOLDER'], 'val'))

#Note that we shuffle the data for the training set, but not for the validation set:
train_loader = DataLoader(dataset=training_dataset, batch_size=EXPERIMENT_CONFIG['batch_size'], shuffle=True, pin_memory=True)
validate_loader = DataLoader(dataset=validate_dataset, batch_size=EXPERIMENT_CONFIG['batch_size'], shuffle=True, pin_memory=True)

print(f'Train samples: {len(training_dataset)}')
print(f'Val samples: {len(validate_dataset)}')

100%|██████████| 120/120 [00:06<00:00, 17.37it/s]

Train samples: 1620
Val samples: 120


# Train

In [5]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: nhikieu (qut_nhi_phd). Use `wandb login --relogin` to force relogin


True

In [6]:
ckpt_path = r"D:\drfuse\checkpoints\1720658579\1720683639_loss2.0603"
with wandb.init(project="drfuse", entity="nhikieu", config=EXPERIMENT_CONFIG):
    config = wandb.config
    model_trainer = DrFuse_RS_Trainer(config, pretrain=True, ckpt_path=ckpt_path)
    model_trainer(train_loader, validate_loader)

wandb: Currently logged in as: nhikieu. Use `wandb login --relogin` to force relogin


  0%|          | 0/100 [00:00<?, ?it/s]d:\drfuse\models\drfuse_rs.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = (images.to(device), labels.to(device), torch.tensor(masks).to(device))
d:\drfuse\drfuse_venv\lib\site-packages\torch\nn\modules\instancenorm.py:80: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
d:\drfuse\models\drfuse_rs.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = (images.to(device), labels.to(device), torch.tensor(masks).to(device))


Train loss after 00512 examples: 3.786
Val loss after 00512 examples: 4.040
Train loss after 01024 examples: 3.984
Val loss after 01024 examples: 3.988
Train loss after 01536 examples: 4.018
Val loss after 01536 examples: 3.965


  1%|          | 1/100 [03:52<6:23:40, 232.54s/it]

Train loss after 01620 examples: 3.619
Val loss after 01620 examples: 3.964
Train loss after 02048 examples: 3.691
Val loss after 02048 examples: 3.935
Train loss after 02560 examples: 3.723
Val loss after 02560 examples: 3.872
Train loss after 03072 examples: 3.761
Val loss after 03072 examples: 3.824
Train loss after 03240 examples: 3.562
Val loss after 03240 examples: 3.808


...

 99%|█████████▉| 99/100 [6:56:24<03:57, 237.84s/it]

Train loss after 160380 examples: 1.781
Val loss after 160380 examples: 2.081
Train loss after 160768 examples: 2.080
Val loss after 160768 examples: 2.103
Train loss after 161280 examples: 1.876
Val loss after 161280 examples: 2.080
Train loss after 161792 examples: 1.902
Val loss after 161792 examples: 2.080


100%|██████████| 100/100 [7:00:18<00:00, 252.19s/it]

Train loss after 162000 examples: 1.779
Val loss after 162000 examples: 2.098


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_loss,██▇▇▆▅▆▇▅▄▅▃▅▄▆▄▃▄▃▃▄▃▃▃▃▂▂▂▄▃▂▃▁▁▂▂▂▂▁▁
val_loss,█▇▇▇▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▃▂▂▂▂▂▂▁▁▁▁▁
epoch,99
train_loss,1.77871
val_loss,2.09775


# Unit Test

In [2]:
dummies = torch.rand(6, 4, 512, 512)
masks = torch.FloatTensor([True, False, True, True, False, False])
model = DrFuse_RS_Model()
results = model(dummies, masks)
print(results['pred_rgb'].shape, results['pred_ndsm'].shape, 
      results['pred_shared'].shape, results['pred_multimodal'].shape,
      results['aux_preds'][0].shape)

# Assuming model is an instance of a class derived from torch.nn.Module
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {total_params}")

/Users/nhikieu/Documents/GitHub/drfuse/drfuse_venv/lib/python3.9/site-packages/torch/nn/modules/instancenorm.py:115: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(


torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512])


In [ ]:
pairs = torch.FloatTensor([True, False, True, False, True, True])
pairs = pairs.unsqueeze(1)
feat_dummy = torch.randn(6, 512)
test = pairs*feat_dummy
test

In [ ]:
for i, (input_4C, gt, mask) in enumerate(validate_loader):
  print(input_4C.shape, gt.shape, mask.shape)
  print(mask)
  break